In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
data = spark.read.table('movielens_ratings_csv')
data.show(5)

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
+-------+------+------+
only showing top 5 rows



In [0]:
data.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [0]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [0]:
training, test = data.randomSplit([0.8,0.2])

In [0]:
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating'
)

In [0]:
model = als.fit(training)

In [0]:
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      3|   1.0|     1|  1.1864222|
|      4|   1.0|     5|  1.5564624|
|      5|   1.0|     5|  0.5766406|
|      2|   3.0|     6|  2.5570943|
|      5|   1.0|     6| 0.73949486|
|      1|   1.0|     7| 0.91071194|
|      4|   1.0|     7| 0.97161025|
|      3|   2.0|     8|  1.9598169|
|      4|   1.0|     9|   4.358823|
|      4|   1.0|    12|-0.08359909|
|      1|   1.0|    14| 0.85626185|
|      1|   4.0|    15|  1.2512476|
|      2|   1.0|    15|  2.9133172|
|      4|   3.0|    18|  0.1116925|
|      1|   1.0|    19|  1.0818468|
|      2|   1.0|    19| 0.25212812|
|      2|   4.0|    21|   0.877497|
|      0|   1.0|    22|     1.3565|
|      3|   2.0|    22| 0.83711183|
|      1|   3.0|    25| 0.26378453|
+-------+------+------+-----------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(
    metricName = 'rmse',
    labelCol = 'rating',
    predictionCol = 'prediction'
)

In [0]:
rmse = evaluator.evaluate(predictions)
print(rmse)

1.6848693678216746


In [0]:
single_user = test.filter(test['userId']==11).select('movieId','userId')
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     12|    11|
|     16|    11|
|     39|    11|
|     40|    11|
|     62|    11|
|     70|    11|
|     72|    11|
|     75|    11|
|     79|    11|
|     90|    11|
|     94|    11|
+-------+------+



In [0]:
recommendations = model.transform(single_user)
recommendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     90|    11|   2.212029|
|     12|    11|   2.024562|
|     11|    11|  1.7601181|
|     70|    11|  1.6213994|
|     16|    11|  1.5207709|
|     94|    11|  1.5180707|
|     75|    11|   1.056796|
|     40|    11| 0.05097246|
|     39|    11|0.004725173|
|     79|    11| -0.9627025|
|     72|    11| -1.1963868|
|     62|    11|  -3.841011|
+-------+------+-----------+

